In [ ]:
!pip install --quiet datasets
import pandas as pd
from datasets import list_datasets,load_dataset
!pip install --quiet transformers
from transformers import BertTokenizerFast,BertForSequenceClassification
from sklearn.model_selection import train_test_split
import torch

     |████████████████████████████████| 325 kB 9.7 MB/s 
     |████████████████████████████████| 136 kB 41.6 MB/s 
     |████████████████████████████████| 1.1 MB 48.1 MB/s 
     |████████████████████████████████| 212 kB 34.4 MB/s 
     |████████████████████████████████| 77 kB 5.6 MB/s 
     |████████████████████████████████| 127 kB 39.9 MB/s 
     |████████████████████████████████| 271 kB 40.5 MB/s 
     |████████████████████████████████| 94 kB 2.3 MB/s 
     |████████████████████████████████| 144 kB 39.5 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 4.0 MB 12.2 MB/s 
     |████████████████████████████████| 596 kB 48.3 MB/s 
     |████████████████████████████████| 895 kB 45.5 MB/s 
     |████████████████████████████████| 6.

Add Required Libraries

Load Dataset

In [ ]:
dataset=load_dataset("multidoc2dial",name="multidoc2dial")

Generating validation split:   0%|          | 0/4201 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/21451 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5 [00:00<?, ? examples/s]

Dataset multi_doc2dial downloaded and prepared to /root/.cache/huggingface/datasets/multi_doc2dial/multidoc2dial/1.0.0/6e2a407c09eb478a5b80845fc04406bb9c9d4bea9f135dd7b8b7610a6b608d84. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Change to dataframe and use dmv subset and perform cleaning


In [ ]:
df=pd.DataFrame(dataset['train'])
dmv=df[df['domain']=='dmv']

In [ ]:
len(dmv)

6135

In [ ]:
dmv.head()

,id,title,context,question,da,answers,utterance,domain
0,8df07b7a98990db27c395cb1f68a962e_1,Top 5 DMV Mistakes and How to Avoid Them#3_0,Many DMV customers make easily avoidable mista...,"Hello, I forgot o update my address, can you h...",query_condition,{'text': ['you must report a change of address...,"hi, you have to report any change of address t...",dmv
1,8df07b7a98990db27c395cb1f68a962e_3,Top 5 DMV Mistakes and How to Avoid Them#3_0,Many DMV customers make easily avoidable mista...,Can I do my DMV transactions online?[SEP]agent...,query_solution,"{'text': ['Sign up or log into MyDMV [6 ] '], ...","Yes, you can sign up for MyDMV for all the onl...",dmv
2,8df07b7a98990db27c395cb1f68a962e_5,Registration suspensions for failure to pay to...,\n\nWhat is a suspension for failure to pay to...,You've got it. Another query about DMV. What h...,query_condition,{'text': ['the suspension is placed on hold pe...,the suspension is placed on hold pending the o...,dmv
3,8df07b7a98990db27c395cb1f68a962e_7,Registration suspensions for failure to pay to...,\n\nWhat is a suspension for failure to pay to...,"Besides that, will I receive a notice?[SEP]age...",query_solution,{'text': ['the NYS Department of Motor Vehicle...,"the NYS Department of Motor Vehicles , "" DMV ...",dmv
4,8df07b7a98990db27c395cb1f68a962e_9,Registration suspensions for failure to pay to...,\n\nWhat is a suspension for failure to pay to...,If you submit the affidavit?[SEP]agent: the N...,query_condition,{'text': ['you will not have to appear at a DM...,you will not have to appear at a DMV hearing,dmv


Preprocess context

In [ ]:
import re
dmv['contextClean']=dmv['context'].apply(lambda x:re.sub(r'\[.*?\]','',x))
dmv['contextClean']=dmv['contextClean'].apply(lambda x:re.sub(r'[^\w\s]','',x))
dmv['contextClean']=dmv['contextClean'].apply(lambda x:re.sub(r'[\n,*-:]','',x))
dmv['contextClean']=dmv['contextClean'].apply(lambda x:re.sub(r'\s+',' ',x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Split question history to get just question 

In [ ]:
dmv['questionSplit']=dmv['question'].str.split("[").apply(lambda x: x[0])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
dmv['contextClean'][2]

'What is a suspension for failure to pay tolls If you do not pay a required toll fee or other charge to a tolling authority which operates a toll highway bridge andor tunnel facility your registration may be suspended Who issues toll violations A tolling authority which includes the New York State Thruway Authority the Triborough Bridge and Tunnel Authority MTA Bridges and Tunnels and the Port Authority of New York and New Jersey What happens if I don t pay a toll If you fail to pay tolls fees or other charges in relation to or more tolling violations all committed within years or fail to pay in tolls accumulated within years if the violations occurred in a commercial vehicle or fail to have them dismissed or transferred the NYS Department of Motor Vehicles DMV may send you a Notice of Registration Suspension The notice will advise you that if you pay all of the tolls fees or other charges owed to the tolling authority or have them dismissed or transferred the suspension will not take 

"You've got it. Another query about DMV. What happens if you seek a hearing with the DMV?[SEP]agent: Yes, you can sign up for MyDMV for all the online transactions needed.||user: Can I do my DMV transactions online?||agent: hi, you have to report any change of address to DMV within 10 days after moving. You should do this both for the address associated with your license and all the addresses associated with all your vehicles.||user: Hello, I forgot o update my address, can you help me with that?"

calculate answer end from answer text

In [ ]:
from tqdm.auto import tqdm
def addEndIDX(answers,contexts):
  new_answers=[]
  for answer, context in tqdm(zip(answers,contexts)):
    answer['text']=answer['text'][0]
    answer['answer_start']=answer['answer_start'][0]
    gold_text=answer['text']
    start_idx=answer['answer_start']
    end_idx=start_idx+len(gold_text)
    answer['answer_end']=end_idx
    if context[start_idx:end_idx]==gold_text:
      answer['answer_end']=end_idx
    else:
        # this means the answer is off by 1-2 tokens
        for n in [1, 2]:
            if context[start_idx-n:end_idx-n] == gold_text:
                answer['answer_start'] = start_idx - n
                answer['answer_end'] = end_idx - n
    new_answers.append(answer)
  return new_answers
def prep_data(dataset):
  questions=dataset['question'].tolist()
  contexts=dataset['context'].tolist()
  answers=addEndIDX(dataset['answers'].tolist(),contexts)
  return{'question':questions,'context':contexts,'answer':answers}


In [ ]:
dmvqa=prep_data(dmv)

0it [00:00, ?it/s]

tokenize bert

In [ ]:
tokenizer=BertTokenizerFast.from_pretrained('bert-base-uncased',max_position_embeddings=2048)
train=tokenizer(dmvqa['context'],dmvqa['question'],padding=True,return_tensors='pt',truncation=True)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
len(dmvqa['context'])

6135

In [ ]:
len(dmvqa['answer'])

6135

Add token positions

In [ ]:
def add_token_positions(encodings,answers):
  start_positions=[]
  end_positions=[]
  for i in tqdm(range(len(answers))):
    start_positions.append(encodings.char_to_token(i,answers[i]['answer_start']))
    end_positions.append(encodings.char_to_token(i,answers[i]['answer_end']))
    if start_positions[-1] is None:
      start_positions[-1]=tokenizer.model_max_length
    go_back=1
    while end_positions[-1] is None:
      end_positions[-1]=encodings.char_to_token(i,answers[i]['answer_end']-go_back)
      go_back+=1
  encodings.update({'start_positions':start_positions,'end_positions':end_positions})
        

In [ ]:
add_token_positions(train,dmvqa['answer'])

  0%|          | 0/6135 [00:00<?, ?it/s]

In [ ]:
train.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'])

In [ ]:
dmv['answers'][1]

{'answer_end': 4527,
 'answer_start': 4496,
 'text': 'Sign up or log into MyDMV [6 ] '}

In [ ]:
train['start_positions'][:5]

[62, 512, 224, 512, 438]

Define dmv dataset

In [ ]:
class dmvDataset(torch.utils.data.Dataset):
  def __init__(self,encodings):
    self.encodings=encodings
  def __getitem__(self,idx):
    return {key:torch.tensor(val[idx]) for key,val in self.encodings.items()}
  def __len__(self):
    return len(self.encodings.input_ids)

In [ ]:
train_dataset=dmvDataset(train)

In [ ]:
train_dataset

Give batch size of 16 to train

In [ ]:
loader=torch.utils.data.DataLoader(train_dataset,batch_size=4,shuffle=True)

In [ ]:
from transformers import BertForQuestionAnswering

Initiate Bert for QA

In [ ]:
model=BertForQuestionAnswering.from_pretrained("bert-base-uncased")

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased a

In [ ]:
def compute_metrics(eval_preds):
   preds, labels = eval_preds
   decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
   # Replace -100 in the labels as we can't decode them.
   labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
   decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
   # Some simple post-processing
   decoded_preds=['\n'.join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]  
   decoded_labels=['\n'.join(nltk.sent_tokenize(labels.strip())) for pred in decoded_labels]
   result=metric.compute(predictions=decoded_preds,references=decoded_labels,use_stemmer=True)
   result={key: value.mid.fmeasure * 100 for key,value in result.items()}
   prediction_lens=[np.count_nonzero(pred!=tokenizer.pad_token_id) for pred in preds]
   result['gen_length']=np.mean(prediction_lens)
   return {k:round(v,4).to(device) for k,v in result.items()}

set model hyperparameters

In [ ]:
from transformers import AdamW
device=torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
model.train()
optim=AdamW(model.parameters(),lr=5e-5)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
for epoch in range(3):
  loop=tqdm(loader)
  for batch in loop:
    optim.zero_grad()
    input_ids=batch['input_ids'].to(device)
    attention_mask=batch['attention_mask'].to(device)
    start_positions=batch['start_positions'].to(device)
    end_positions=batch['end_positions'].to(device)
    outputs=model(input_ids,attention_mask=attention_mask,start_positions=start_positions,end_positions=end_positions)
    loss=outputs[0]
    loss.backward()
    optim.step()
    loop.set_description(f'Epoch {epoch}')
    loop.set_postfix(loss=loss.item())

  0%|          | 0/1534 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """


  0%|          | 0/1534 [00:00<?, ?it/s]

  0%|          | 0/1534 [00:00<?, ?it/s]

Save model and tokenizer

In [ ]:
model.save_pretrained('./bert-qa')

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
tokenizer.save_pretrained('./token_bert-qa')

('./token_bert-qa/tokenizer_config.json',
 './token_bert-qa/special_tokens_map.json',
 './token_bert-qa/vocab.txt',
 './token_bert-qa/added_tokens.json',
 './token_bert-qa/tokenizer.json')

In [ ]:
dmv['question'][0]

'Hello, I forgot o update my address, can you help me with that?[SEP]'

In [ ]:
dmv['contextClean'][0]

'Many DMV customers make easily avoidable mistakes that cause them significant problems including encounters with law enforcement and impounded vehicles Because we see customers make these mistakes over and over again we are issuing this list of the top five DMV mistakes and how to avoid them Forgetting to Update Address By statute you must report a change of address to DMV within ten days of moving That is the case for the address associated with your license as well as all the addresses associated with each registered vehicle which may differ It is not sufficient to only write your new address on the back of your old license tell the United States Postal Service or inform the police officer writing you a ticket If you fail to keep your address current you will miss a suspension order and may be charged with operating an unregistered vehicle andor aggravated unlicensed operation both misdemeanors This really happens but the good news is this is a problem that is easily avoidable Learn

Testing

In [ ]:
question = dmv['questionSplit'][0]
context = dmv['contextClean'][0]
inputs = tokenizer.encode_plus(question, context, return_tensors="pt",max_length=512).to(device) 
answer_start_scores, answer_end_scores = model(**inputs,return_dict=False)
answer_start = torch.argmax(answer_start_scores)  # get the most likely beginning of answer with the argmax of the score
answer_end = torch.argmax(answer_end_scores) + 1  # get the most likely end of answer with the argmax of the score

ans=tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end]))

In [ ]:
dmv['answers'][0]['text']

'you must report a change of address to DMV within ten days of moving. That is the case for the address associated with your license, as well as all the addresses associated with each registered vehicle, which may differ. '

In [ ]:
import nltk
from nltk.translate import bleu
from nltk.translate.bleu_score import SmoothingFunction
smoothie = SmoothingFunction().method4


In [ ]:
output=pd.DataFrame(columns=['query','context','ans','exp_ans','bleu_score'])

Calculate Bleu Score

In [ ]:
for i in range(len(dmv)):
  question = dmv['question'][i]
  context = dmv['contextClean'][i]
  inputs = tokenizer.encode_plus(question, context, return_tensors="pt",max_length=512).to(device) 
  answer_start_scores, answer_end_scores = model(**inputs,return_dict=False)
  answer_start = torch.argmax(answer_start_scores)  # get the most likely beginning of answer with the argmax of the score
  answer_end = torch.argmax(answer_end_scores) + 1  # get the most likely end of answer with the argmax of the score

  ans=tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end]))
  output=output.append({'query':dmv['question'][i],'context':dmv['contextClean'][i],'ans':ans,'exp_ans':dmv['answers'][i]['text'],'bleu_score':bleu([ans], dmv['answers'][i]['text'], smoothing_function=smoothie)},ignore_index=True)

In [ ]:
output.to_csv('QAResultsQuestion.csv')

In [ ]:
output.head()

,query,context,ans,exp_ans,bleu_score
0,"Hello, I forgot o update my address, can you h...",Many DMV customers make easily avoidable mista...,you help me with that? [SEP] [SEP] many dmv cu...,you must report a change of address to DMV wit...,0.163450
1,Can I do my DMV transactions online?[SEP]agent...,Many DMV customers make easily avoidable mista...,,Sign up or log into MyDMV [6 ],0.000000
2,You've got it. Another query about DMV. What h...,What is a suspension for failure to pay tolls ...,,the suspension is placed on hold pending the o...,0.000000
3,"Besides that, will I receive a notice?[SEP]age...",What is a suspension for failure to pay tolls ...,"besides that, will i receive a notice? [SEP] a...","the NYS Department of Motor Vehicles, "" DMV "" ...",0.013074
4,If you submit the affidavit?[SEP]agent: the N...,What is a suspension for failure to pay tolls ...,"besides that, will i receive a notice? | | age...",you will not have to appear at a DMV hearing.,0.146558
